# Nuclear potential for nucleons; reproducing binding energy
2024-04-22

Process:
- DONE define the potential
- DONE construct the many-particle potential
- DONE trial minimise for multiple nucleons, obtain binding energy
- test on some more large atoms
- visualise to check behaviour

In [2]:
import numpy as np
import pandas as pd

import jax
import jax.numpy as jnp

import plotly.express as px

In [3]:
eps0 = 8.85418782e-12
e = 1.60217662e-19
c = 299792458.0

# reduced units
rc = 1e-15
ec = 1.60217662e-19
mc = 1.6726219e-27

# compute constant before elmag potential in reduced units
e**2 / (4*np.pi*eps0) / ec / rc # eV

1439964.5340890521

## Define functions

In [4]:
def elmag_potential(r, t1, t2):
    """In MeV, r in fm"""
    return 1.44 * t1 * t2 / r

def elmag_potential_jnp(r, t1, t2):
    """In MeV, r in fm"""
    return 1.44 * t1 * t2 / r

def strong_potential(r, V0=50.0, r0=1.25, a=0.2):
    """In MeV, r in fm
    r0 should be 1.25 * A**(1/3)
    """
    return -V0 / (1.0 + np.exp((r - r0) / a))

def strong_potential_jnp(r, V0=50.0, r0=1.25, a=0.2):
    """In MeV, r in fm
    r0 should be 1.25 * A**(1/3)
    """
    return -V0 / (1.0 + jnp.exp((r - r0) / a))

In [5]:
r = np.linspace(0.01, 5, 1000)
V = strong_potential(r, a=0.1)

# px.line(x=r, y=V)

In [6]:
def total_energy(R, T):
    V = 0.0
    for i, _ in enumerate(T):
        for j in range(i):
            r = np.linalg.norm(R[i] - R[j])
            V += elmag_potential(r, T[i], T[j]) + strong_potential(r)
    return V

def total_energy_jnp(R, T):
    V = 0.0
    for i, _ in enumerate(T):
        for j in range(i):
            r = jnp.linalg.norm(R[i] - R[j])
            V += elmag_potential_jnp(r, T[i], T[j]) + strong_potential_jnp(r)
    return V

## Simulate an atom

In [59]:
Z = 2
A = 4
N = A - Z

T = np.array([1] * Z + [0] * N)

In [22]:
# jax
# define random seed in jax
# key = jax.random.PRNGKey(0)

In [50]:
mc * c**2 / ec / 1e6

938.2720830701021

In [32]:
total_energy(R, T)

-240.56042307949724

### Minimize

In [ ]:
# numpy functions with hard-coded gradient

# def elmag_force(R, T):
#     F = np.zeros_like(R)
#     for i, _ in enumerate(T):
#         for j in range(i):
#             r = R[i] - R[j]
#             F[i] += elmag_potential(r, T[i], T[j]) * r / np.linalg.norm(r)**3
#     return F

# def strong_force(R, T):
#     F = np.zeros_like(R)
#     for i, _ in enumerate(T):
#         for j in range(i):
#             r = R[i] - R[j]
#             F[i] += strong_potential(r) * r / np.linalg.norm(r)**3
#     return F

# def grad(R, T):
#     grad = np.zeros_like(R)
#     for i, _ in enumerate(T):
#         for j in range(i):
#             r = np.linalg.norm(R[i] - R[j])
#             grad[i] += (R[i] - R[j]) / r * (elmag_potential(r, T[i], T[j]) + strong_potential(r))
#     return grad

In [82]:
def minimize_energy(R, T, n_steps=10, lr=0.01, verbose=False, thermo=10):
    """Gradient descent for minimisation"""
    for i in range(n_steps):
        R -= lr * jax.grad(total_energy_jnp)(R, T)
        if verbose and i % thermo == 0:
            print(i, total_energy_jnp(R, T))
    return R

In [56]:
np.random.seed(42)

R = (np.random.rand(A, 3) - 0.5)  # FINE TUNE

In [57]:
R

array([[-0.12545988,  0.45071431,  0.23199394],
       [ 0.09865848, -0.34398136, -0.34400548],
       [-0.44191639,  0.36617615,  0.10111501],
       [ 0.20807258, -0.47941551,  0.46990985]])

In [64]:
total_energy_jnp(R, T)

Array(-294.6174, dtype=float32)

In [65]:
jax.grad(total_energy_jnp)(R, T)

Array([[ 0.0250228 , -0.06546056, -0.10656619],
       [ 0.02206689, -0.05801904, -0.09404802],
       [-0.15101355,  0.19575506, -0.42411137],
       [ 0.10392384, -0.0722754 ,  0.6247256 ]], dtype=float32)

In [66]:
R = minimize_energy(R, T, verbose=True)

0 -294.62067
1 -294.61865
2 -294.62064
3 -294.6192
4 -294.62045
5 -294.61948
6 -294.6203
7 -294.61966
8 -294.62018
9 -294.61975


Another trial: carbon

In [80]:
Z, A = 6, 12
N = A - Z

T = np.array([1] * Z + [0] * N)

np.random.seed(43)

R = (np.random.rand(A, 3) - 0.5)  # FINE TUNE

In [81]:
%%time

R = minimize_energy(R, T, n_steps=30, lr=1e-3, verbose=True)

0 -3170.0957 -11259268166.93725
10 -3215.625 -11259268212.466331
20 -3219.106 -11259268215.947285


In [91]:
# plot
df = pd.DataFrame(R, columns=["x", "y", "z"], index=range(1, A+1))
df['type'] = T
df['type'] = df['type'].astype(str)

In [92]:
fig = px.scatter_3d(df, x="x", y="y", z="z", color='type', opacity=0.7)
fig.show()

Final trial: aluminium

In [97]:
Z, A = 13, 27
N = A - Z

T = np.array([1] * Z + [0] * N)

np.random.seed(43)

R = (np.random.rand(A, 3) - 0.5)

In [98]:
%%time

R = minimize_energy(R, T, n_steps=30, lr=1e-3, verbose=True, thermo=2)

0 -16800.912
2 -17070.629
4 -17101.766
6 -17104.568
8 -17106.053
10 -17106.68
12 -17106.844
14 -17106.988
16 -17107.137
18 -17107.275
20 -17107.426
22 -17107.562
24 -17107.688
26 -17107.74
28 -17107.797
CPU times: user 1min 52s, sys: 877 ms, total: 1min 52s
Wall time: 2min 9s


In [106]:
# plot
df = pd.DataFrame(R, columns=["x", "y", "z"], index=range(1, A+1))
df['type'] = T
df['type'] = df['type'].astype(str)
# df['size'] = 100

fig = px.scatter_3d(df, x="x", y="y", z="z", color='type', opacity=0.7)
fig.show()

### Learnings
- all neutrons collapsed in the centre
- all protons are distributed at some distance from the centre as if on a sphere
- hence, strong force needs some internal barrier (repulsive core) if it is supposed to represent reality

## Nuclear force with repulsive core

In [11]:
def strong_potential_jnp(r, epsilon=50.0, sigma=1/2**(1/6)):
    """In MeV, r in fm"""
    return 4 * epsilon * ((sigma / r) ** 12 - (sigma / r) ** 6)

def total_energy_jnp(R, T):
    V = 0.0
    for i, _ in enumerate(T):
        for j in range(i):
            r = jnp.linalg.norm(R[i] - R[j])
            V += elmag_potential_jnp(r, T[i], T[j]) + strong_potential_jnp(r)
    return V

def minimize_energy(R, T, n_steps=10, lr=0.01, verbose=False, thermo=10):
    """Gradient descent for minimisation"""
    for i in range(n_steps):
        R -= lr * jax.grad(total_energy_jnp)(R, T)
        if verbose and i % thermo == 0:
            print(i, total_energy_jnp(R, T))
    return R

Trial atom

In [109]:
Z, A = 2, 5
N = A - Z

T = np.array([1] * Z + [0] * N)

np.random.seed(43)

R = (np.random.rand(A, 3) - 0.5) * 1.5

In [114]:
%%time

R = minimize_energy(R, T, n_steps=20, lr=1e-4, verbose=True, thermo=2)

0 -149.3809
2 -149.9536
4 -149.99582
6 -149.99792
8 -149.99802
10 -149.99803
12 -149.99805
14 -149.99803
16 -149.99802
18 -149.99805
CPU times: user 2.45 s, sys: 33.1 ms, total: 2.48 s
Wall time: 3.02 s


In [115]:
R.astype(int)

Array([[   0,    0,    0],
       [ -37,  730,  -19],
       [   0,    0,    0],
       [   0,    0,    0],
       [  37, -731,   20]], dtype=int32)

In [108]:
# plot
df = pd.DataFrame(R, columns=["x", "y", "z"], index=range(1, A+1))
df['type'] = T
df['type'] = df['type'].astype(str)

fig = px.scatter_3d(df, x="x", y="y", z="z", color='type', opacity=0.7)
fig.show()

In [105]:
d_res = {
    (1, 2): -50,
    (1, 3): -150,
    (2, 4): -223.82048,
}